In [239]:
import pandas as pd
import numpy as np

# Part I

In [240]:
def hash_map(x):
    value = 0
    for char in list(x.strip()):
        value += ord(char)
        value *= 17
        value %= 256
    return value

In [241]:
se = pd.read_csv("example_1.txt", header=None).T[0]
print(se.values)

['HASH']


In [242]:
se.apply(hash_map).sum()

52

In [243]:
se = pd.read_csv("example_2.txt", header=None).T[0]
result = se.apply(hash_map)
pd.merge(se, result, left_index=True, right_index=True)

,0_x,0_y
0,rn=1,30
1,cm-,253
2,qp=3,97
3,cm=2,47
4,qp-,14
5,pc=4,180
6,ot=9,9
7,ab=5,197
8,pc-,48
9,pc=6,214


In [244]:
result.sum()

1320

In [245]:
se = pd.read_csv("input.txt", header=None).T[0]
# se.apply(hash_map).sum()

# Part II

In [246]:
se = pd.read_csv("example_2.txt", header=None).T[0]
steps = se.str.extract("(?P<Label>.*)(?P<Operation>=|-)(?P<FocalLength>\d*)")
steps["Box"] = steps["Label"].apply(hash_map)
steps

,Label,Operation,FocalLength,Box
0,rn,=,1,0
1,cm,-,,0
2,qp,=,3,1
3,cm,=,2,0
4,qp,-,,1
5,pc,=,4,3
6,ot,=,9,3
7,ab,=,5,3
8,pc,-,,3
9,pc,=,6,3


In [247]:
def perform_lens_operation(x):
    if x["Operation"] == "-":
        if x["Box"] in boxes.keys() and x["Label"] in boxes[x["Box"]].keys():
            boxes[x["Box"]].pop(x["Label"])
    else:
        if not x["Box"] in boxes:
            boxes[x["Box"]] = {}
        boxes[x["Box"]][x["Label"]] = x["FocalLength"]


boxes = {}
steps.apply(perform_lens_operation, axis="columns")
boxes

{0: {'rn': '1', 'cm': '2'}, 1: {}, 3: {'ot': '7', 'ab': '5', 'pc': '6'}}

In [248]:
focal_length = pd.DataFrame(boxes).fillna(0).astype(int)
focal_length

,0,1,3
rn,1,0,0
cm,2,0,0
ot,0,0,7
ab,0,0,5
pc,0,0,6


In [249]:
box_power = np.array(focal_length.columns) + 1
box_power

array([1, 2, 4], dtype=int64)

In [250]:
slot_power = (focal_length > 0).cumsum()
slot_power

,0,1,3
rn,1,0,0
cm,2,0,0
ot,2,0,1
ab,2,0,2
pc,2,0,3


In [251]:
((focal_length * slot_power).sum() * box_power).sum()

145